# 3 TensorFlow 入门

## 3.1 TensorFlow 计算模型——计算图

In [1]:
import tensorflow as tf

a = tf.constant([1.0, 2.0], name="a")
b = tf.constant([2.0, 3.0], name="b")
result = a + b

sess = tf.Session()
sess.run(result)

array([ 3.,  5.], dtype=float32)

In [22]:
# 获取当前默认的计算图。
print(a.graph is tf.get_default_graph()) 

True


In [23]:
# 使用Graph生成新的计算图。不同计算图上的张量和运算不会共享
import tensorflow as tf

print(tf.__version__)
print(tf.__path__)

g1 = tf.Graph()
with g1.as_default():
    # 0.9.0版本可用，新版本报错：TypeError: __init__() got an unexpected keyword argument 'shape'
#     v = tf.get_variable("v", initializer=tf.zeros_initializer(shape=[1])) 
    v = tf.get_variable("v", initializer=tf.zeros_initializer()(shape=[1])) # 初始化为0
    
g2 = tf.Graph()
with g2.as_default():
    v= tf.get_variable("v", initializer=tf.ones_initializer()(shape=[1])) # 初始化为1

with tf.Session(graph=g1) as sess:
    tf.initialize_all_variables().run() 
    with tf.variable_scope("", reuse=True):
        print(sess.run(tf.get_variable("v")))
        
with tf.Session(graph=g2) as sess:
    # WARNING: use tf.global_variables_initializer instead
#     tf.initialize_all_variables().run()
    init = tf.global_variables_initializer()
    sess.run(init)
    with tf.variable_scope("", reuse=True):
        print(sess.run(tf.get_variable("v")))

1.4.1
['/usr/local/lib/python2.7/dist-packages/tensorflow']
[ 0.]
[ 1.]


In [24]:
# 指定运行设备
g = tf.Graph()
with g.device("/gpu:0"):
    result = a + b
    print(result)

Tensor("add_4:0", shape=(2,), dtype=float32)


## 3.2 TensorFlow 数据模型——张量

In [25]:
import tensorflow as tf
a = tf.constant([1.0, 2.0], name="a")
b = tf.constant([2.0, 3.0], name="b")
result = tf.add(a, b, name="result")
print(result) # 名字、维度、类型

Tensor("result:0", shape=(2,), dtype=float32)


In [32]:
# a = tf.constant([1, 2], name="a")
a = tf.constant([1, 2], name="a", dtype=tf.float32)
b = tf.constant([2.0, 3.0], name="b") 

# ValueError: Tensor conversion requested dtype int32 for Tensor with dtype float32:'Tensor("b_3:0", shape=(2,), dtype=float32)'
result = a + b

## 3.3 TensorFlow 运行模型——会话

In [29]:
sess = tf.Session()
with sess.as_default(): # 设置默认会话
    print(result.eval()) 

[ 3.  5.]


In [33]:
sess = tf.Session()
print(sess.run(result))
print(result.eval(session=sess)) # 指定会话

[ 3.  5.]
[ 3.  5.]


In [34]:
sess = tf.InteractiveSession() # 直接构建默认会话
print(result.eval())
sess.close()

[ 3.  5.]


In [ ]:
# 配置会话，设置不支持GPU时自动调整到CPU，日志中记录每个节点被安排了哪个设备
config = tf.ConfigProto(allow_soft_placement=True,log_device_placement=True)
sess1 = tf.InteractiveSession(config=config)
sess2 = tf.Session(config=config)

## 3.4 Tensorflow 实现神经网络

In [6]:
import tensorflow as tf

# stddev方差，mean默认0，seed随机种子
w1 = tf.Variable(tf.random_normal([2, 3], stddev=1, seed=1))
w2 = tf.Variable(tf.random_normal([3, 1], stddev=1, seed=1))

# 1*2 矩阵
x = tf.constant([[0.7, 0.9]])

# 前向传播
a = tf.matmul(x, w1); # a=x*w1
y = tf.matmul(a, w2); # y=a*w2

sess = tf.Session()
# 变量初始化
# sess.run(w1.initializer)
# sess.run(w2.initializer)
init_op = tf.global_variables_initializer()
sess.run(init_op)

print(sess.run(y))
sess.close()

[[ 3.95757794]]


In [13]:
# 维度和类型
w1 = tf.Variable(tf.random_normal([2, 3], stddev=1), name="w1")
w2 = tf.Variable(tf.random_normal([2, 2], stddev=1), name="w2")

# 维度不匹配
# tf.assign(w1, w2)
tf.assign(w1, w2, validate_shape=False)

<tf.Tensor 'Assign_6:0' shape=(2, 2) dtype=float32_ref>

In [15]:
import tensorflow as tf

w1 = tf.Variable(tf.random_normal([2, 3], stddev=1))
w2 = tf.Variable(tf.random_normal([3, 1], stddev=1))

# 占位机制
x = tf.placeholder(tf.float32, shape=(1, 2), name="input")
a = tf.matmul(x, w1);
y = tf.matmul(a, w2);

sess = tf.Session()
init_op = tf.global_variables_initializer()
sess.run(init_op)

# print(sess.run(y))
print(sess.run(y, feed_dict={x: [[0.7, 0.9]]}))

[[ 0.05582702]]


In [23]:
# 完整神经网络样例
import tensorflow as tf
from numpy.random import RandomState

# batch大小
batch_size = 8

w1 = tf.Variable(tf.random_normal([2, 3], stddev=1, seed=1))
w2 = tf.Variable(tf.random_normal([3, 1], stddev=1, seed=1))

x = tf.placeholder(tf.float32, shape=(None, 2), name="x-input")
y_ = tf.placeholder(tf.float32, shape=(None, 1), name="y-input")

# 前向传播
a = tf.matmul(x, w1)
y = tf.matmul(a, w2)

# 损失函数：交叉熵。clip_by_value可以将一个张量的数值限制在范围内，避免类似log0的问题
cross_entropy = -tf.reduce_mean(y_ * tf.log(tf.clip_by_value(y, 1e-10, 1.0)))
# 反向传播算法：梯度下降
train_step = tf.train.AdamOptimizer(0.001).minimize(cross_entropy)

# 通过随机数生成模拟数据集
rdm = RandomState(1)
dataset_size = 128
X = rdm.rand(dataset_size, 2)
# 满足x1+x2<1的样例为正样本（1），否则为负样本（0） 
Y = [[int(x1 + x2 < 1)] for (x1, x2) in X]

with tf.Session as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    print sess.run(w1)
    print sess.run(w2)
    
    STEPS = 5000
    for i in range(STEPS):
        # 每次选取 batch_size 个样本进行训练
        start = (i * batch_size) % dataset_size
        end = min(start + batch_size, dataset_size)
        
        sess.run(train_step, feed_dict={x: X[start, end], y_: Y[start, end]})
        
        if i % 1000 == 0:
            total_cross_entropy = sess.run(cross_entropy, feed_dict={x: X, y_: Y})
            print("After %d trainning step(s), cross entropy on all data is %g" %(i, total_cross_entropy))
            
    print sess.run(w1)
    print sess.run(w2)

AttributeError: __exit__